# Evaluación de Modelos de Clasificación usando SMOTE-NC

Este notebook evalúa el rendimiento de diferentes clasificadores aplicando SMOTE-NC para balancear un dataset desbalanceado. La evaluación incluye métricas como Accuracy, Recall, F1-Score, AUC-ROC, Specificity, y MCC.

## 1. Carga de Datos y Configuración Inicial
Se cargan los datos de entrenamiento y prueba, y se identifican las columnas categóricas necesarias para SMOTE-NC.

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [5]:
# Cargar los conjuntos de entrenamiento y prueba
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

In [6]:
categorical_columns = ['Gender', 'Smoking', 'Hx Smoking', 'Hx Radiothreapy', 'Thyroid Function', 
                       'Physical Examination', 'Adenopathy', 'Pathology', 'Focality', 'Risk', 
                       'T', 'N', 'M', 'Stage', 'Response']

In [7]:
# Obtener índices de columnas categóricas
categorical_indices = [X_train.columns.get_loc(col) for col in categorical_columns]
print(f"Índices de las columnas categóricas: {categorical_indices}")

Índices de las columnas categóricas: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


## 2. Optimización de Hiperparámetros de SMOTE-NC
Usamos `GridSearchCV` para encontrar los mejores parámetros del modelo SMOTE-NC basado en AUC-ROC.

In [8]:
from imblearn.over_sampling import SMOTENC
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score

In [9]:
# Asegurar que las variables objetivo sean unidimensionales
y_train = y_train.to_numpy().ravel()
y_test = y_test.to_numpy().ravel()

In [10]:
# Configurar SMOTE-NC y pipeline con Logistic Regression
smote_nc = SMOTENC(categorical_features=categorical_indices, random_state=42)
log_reg = LogisticRegression(max_iter=1000, random_state=42)
pipeline = make_pipeline(smote_nc, log_reg)

In [11]:
# Hiperparámetros para optimizar
param_grid_smotenc = {
    'smotenc__k_neighbors': [3, 5, 7, 9],
    'smotenc__sampling_strategy': [0.5, 0.7, 1.0]
}

In [12]:
# GridSearchCV para encontrar mejores parámetros
grid_search = GridSearchCV(pipeline, param_grid_smotenc, cv=5, scoring=make_scorer(roc_auc_score), n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('smotenc',
                                        SMOTENC(categorical_features=[1, 2, 3,
                                                                      4, 5, 6,
                                                                      7, 8, 9,
                                                                      10, 11,
                                                                      12, 13,
                                                                      14, 15],
                                                random_state=42)),
                                       ('logisticregression',
                                        LogisticRegression(max_iter=1000,
                                                           random_state=42))]),
             n_jobs=-1,
             param_grid={'smotenc__k_neighbors': [3, 5, 7, 9],
                         'smotenc__sampling_strategy': [0.5, 0.7, 1.0]},
             scoring=make_scorer(roc_auc_score), verbose=1)

In [13]:
# Imprimir los mejores parámetros
print("Mejores parámetros para SMOTE-NC:")
print(grid_search.best_params_)

Mejores parámetros para SMOTE-NC:
{'smotenc__k_neighbors': 5, 'smotenc__sampling_strategy': 1.0}


In [14]:
# Evaluar el modelo en el conjunto de prueba
y_pred_prob = grid_search.predict_proba(X_test)[:, 1]
auc_roc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC en el conjunto de prueba: {auc_roc:.4f}")

AUC-ROC en el conjunto de prueba: 0.9628


## 3. Evaluación de Clasificadores con Datos Sintéticos
Usamos los datos generados con los mejores parámetros de SMOTE-NC para entrenar clasificadores y evaluar su desempeño en el conjunto de prueba original.

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef

In [16]:
# Generar datos sintéticos con los mejores parámetros
best_smote_nc = SMOTENC(categorical_features=categorical_indices, k_neighbors=grid_search.best_params_['smotenc__k_neighbors'],
                        sampling_strategy=grid_search.best_params_['smotenc__sampling_strategy'], random_state=42)
X_synthetic, y_synthetic = best_smote_nc.fit_resample(X_train, y_train)

In [17]:
# Definir clasificadores
classifiers = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'MLP': MLPClassifier(random_state=42, max_iter=1000),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

In [18]:
# Lista para guardar los resultados
results = []

In [19]:
# Entrenar y evaluar clasificadores
for name, clf in classifiers.items():
    clf.fit(X_synthetic, y_synthetic)
    y_pred = clf.predict(X_test)
    y_pred_prob = clf.predict_proba(X_test)[:, 1]
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_pred_prob)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)
    mcc = matthews_corrcoef(y_test, y_pred)
    
    # Guardar resultados
    results.append({
        'Modelo': name,
        'Accuracy': accuracy,
        'Recall': recall,
        'F1-Score': f1,
        'AUC-ROC': auc_roc,
        'Specificity': specificity,
        'MCC': mcc
    })

In [20]:
# Convertir resultados a DataFrame y guardar
results_df = pd.DataFrame(results)
results_df.to_csv('resultados_clasificadores_smote_nc.csv', index=False)
results_df

,Modelo,Accuracy,Recall,F1-Score,AUC-ROC,Specificity,MCC
0,Random Forest,0.961039,0.863636,0.926829,0.986364,1.000000,0.904967
1,MLP,0.909091,0.909091,0.851064,0.975207,0.909091,0.789352
2,Logistic Regression,0.922078,0.909091,0.869565,0.962810,0.927273,0.815726
